In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


init workflow


In [7]:
BoSSSshell.WorkflowMgm.Init("XESF_WedgeFlow_FluxStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Project name is set to 'XESF_WedgeFlow_FluxStudy'.
Default Execution queue is chosen for the database.
Opening existing database '\\dc3\userspace\sebastian\cluster\XESF_WedgeFlow_FluxStudy'.


{ Session Count = 12; Grid Count = 12; Path = \\dc3\userspace\sebastian\cluster\XESF_WedgeFlow_FluxStudy }

In [8]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2024-03-22 17:10:48Z

In [9]:
var sessions = database.Sessions;
sessions

#0: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb1-iFlx3-wFLx0-bFlx0-Fphi0	3/22/2024 5:13:12 PM	d68ad918...
#1: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb1-iFlx0-wFLx0-bFlx0-Fphi0	3/22/2024 5:12:00 PM	3594ef9a...
#2: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb1-iFlx1-wFLx0-bFlx0-Fphi0	3/22/2024 5:12:25 PM	b81c6c3a...
#3: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb1-iFlx2-wFLx0-bFlx0-Fphi0	3/22/2024 5:12:47 PM	f224fa63...
#4: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb2-iFlx2-wFLx0-bFlx0-Fphi0	3/22/2024 5:14:21 PM	7b7ecd83...
#5: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0	3/22/2024 5:11:36 PM	a40f23e7...
#6: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb2-iFlx3-wFLx0-bFlx0-Fphi0	3/22/2024 5:14:47 PM	ff5db0d2...
#7: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb2-iFlx0-wFLx0-bFlx0-Fphi0	3/22/2024 5:13:35 PM	e95c7d39...
#8: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0	3/22/2024 5:13:59 PM	9e59e090...
#9: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPr

In [10]:
MatrixAssembler aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;
si.ID

d68ad918-b0d7-4fe6-8931-2f8026dc6cd7

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(c_P -1)(\rho E - k),~k=c_P \text{Ma}^2\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$


For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the BowShock presented, the v-velocity is zero and we have
$$ h_{in} = \frac{\gamma}{\gamma-1} + \frac{1}{2}u_{in}^2 $$
We prescribe the inflow conditions in terms of pressure $p_{in}=1$, density $\rho_{in}=1$ and the Mach number $M_{in}=4$. From those the velocity is computed as 
$$ u_{in} = M_{in} \sqrt{\gamma \frac{p_{in}}{\rho_{in}}}= 4\sqrt{\gamma}$$

In [11]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 2.0;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

6.300000000000001

In [12]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

## Choose which runs to plot

In [16]:
//var sessions = database.Sessions.Where(s => s.Timesteps.Count >0); //plot all 
var ses=sessions.Where(s => (s.SuccessfulTermination &&s.Name.Contains("iPrb0"))); // the one plotted in the publication
ses


#0: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0	3/22/2024 5:11:36 PM	a40f23e7...
#1: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/22/2024 5:10:24 PM	c1ffd01c...
#2: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0	3/22/2024 5:11:10 PM	9d956eba...
#3: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	3/22/2024 5:10:48 PM	df3f9d42...


# A Helper Function to obtain different plot formats

In [17]:
using BoSSS.Solution.Tecplot;
using System.IO;
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Actual Etnhalpy Error Plot


In [18]:
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    foreach(var timestep in si.Timesteps){
        var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
        enthalpyerrors.Add(error);
    }
    allEE.Add(enthalpyerrors);
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Pick(enthalpyerrors.Count -1)).GetInternal();
    plot.AddDataGroup(si.Name,lts.TimeStepNumbers,enthalpyerrors, GetFormat(count));
    count++;
    enthalpyerrors.SaveToTextFile($"EnthalpyErrors_{si.Name}.txt");
}
plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -16 
 
 
 
 
 10 -14 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 10 0 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 35 
 
 
 
 
 40 
 
 
 
 
 45 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M544.5,57.1 L597.9,57.1 M53.9,50.9 L78.3,52.8 L102.6,53.2 L127.0,54.1 L151.3,63.6 L175.7,67.6
 L200.1,77.1 L224.4,78.0 L248.8,87.6 L273.1,89.4 L297.5,93.4 L321.9,97.3 L346.2,128.0 L370.6,137.9
 L394.9,146.6 L419.3,163.0 L443.7,225.9 L468.0,197.7 L492.4,201.5 L516.7,221.1 L541.1,220.9 L565.5,219.5
 L589.8,203.6 L614.2,204.1 L638.5,204.2 L662.9,204.5 L687.3,205.1 L711.6,206.5 L736.0,206.6 L760.3,206.6
 L784.7,204.9 L809.1,205.6 L833.4,205.7 L857.8,206.1 L882.1,206.4 L906.5,206.6 L930.9,206.8 L955.2,206.8
 L979.6,206.8 L1003.9,206.8 L1028.3,206.8 L1052.7,206.8 L1077.0,206.8 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0

## Residual History

In [19]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    lts.ResHistory.SaveToTextFile($"ResHistory_{si.Name}.txt");
    lts.ResHistory.SaveToTextFile($"EnResHistory_{si.Name}.txt");
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    count++;
    plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -7 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 35 
 
 
 
 
 40 
 
 
 
 
 45 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M55.7,57.1 L109.1,57.1 M45.6,66.2 L61.3,68.8 L76.9,69.4 L92.6,70.5 L108.2,81.7 L123.9,89.8
 L139.5,101.6 L155.2,102.4 L170.8,117.6 L186.5,120.0 L202.2,120.2 L217.8,129.5 L233.5,133.3 L249.1,150.1
 L264.8,163.9 L280.4,191.1 L296.1,208.4 L311.7,216.0 L327.4,219.6 L343.1,221.0 L358.7,221.6 L374.4,222.8
 L390.0,246.8 L405.7,247.3 L421.3,247.4 L437.0,247.6 L452.6,248.1 L468.3,250.7 L484.0,250.8 L499.6,250.8
 L515.3,254.4 L530.9,254.8 L546.6,254.9 L562.2,255.3 L577.9,255.5 L593.5,265.4 L609.2,265.7 L624.9,265.8
 L640.5,265.8 L656.2,265.8 L671.8,265.8 L687.5,265.8 L703.1,265.8 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M55.7,81.1 L109.1,81.1 M45.6,53.2 L61.3,55.9 L76.9,56.5 L92.6,57.4 L108.2,67.5 L123.9,75.6
 L139.5,84.9 L155.2,85.8 L170.8,92.7 L186.5,94.6 L202.2,96.4 L217.8,101.6 L233.5,108.9 L249.1,120.9
 L264.8,135.0 L280.4,174.8 L296.1,186.8 L311.7,181.8 L327.4,185.5 L343.1,185.2 L358.7,185.5 L374.4,185.7
 L390.0,182.8 L405.7,183.1 L421.3,183.2 L437.0,183.3 L452.6,183.5 L468.3,184.2 L484.0,184.2 L499.6,184.2
 L515.3,183.3 L530.9,183.7 L546.6,183.7 L562.2,183.8 L577.9,183.8 L593.5,183.7 L609.2,183.8 L624.9,183.8
 L640.5,183.8 L656.2,183.8 L671.8,183.8 L687.5,183.8 L703.1,183.8 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGWS-15x10-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGWS-15x10-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0

### Plot ShadowFields

In [20]:
BoSSS.Solution.Application.DeleteOldPlotFiles();
foreach(var si in ses){
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    var flds=lts.GetShadowFields();
    var tp= new Tecplot(flds.Last().GridDat,2);
    tp.PlotFields(si.Name,0,flds);
}

rm XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0.plt XDGBS-p3-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0.plt XDGBS-p3-10x32-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0.plt XDGBS-p3-10x32-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0.plt;
